Installation Packages

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import pvlib
from pvlib.forecast import GFS, NAM, NDFD, HRRR, RAP

In [3]:
from pvlib.forecast import GFS

/Users/basilebouquet/Documents/Python/PV_FX/lib/python2.7/site-packages/pvlib/forecast.py:22: UserWarning: The forecast module algorithms and features are highly experimental. The API may change, the functionality may be consolidated into an io module, or the module may be separated into its own package.
  'module, or the module may be separated into its own package.')


In [4]:
import seaborn as sns; sns.set_color_codes()

In [5]:
from pvlib.forecast import GFS, NAM, NDFD, HRRR, RAP

In [6]:
latitude, longitude, tz = 32.2, -110.9, 'US/Arizona'

In [7]:
start = pd.Timestamp(datetime.date.today(), tz=tz)
end = start + pd.Timedelta(days=7)
irrad_vars = ['ghi', 'dni', 'dhi']

In [8]:
model = GFS()
raw_data = model.get_data(latitude, longitude, start, end)
print(raw_data.head())

                           Downward_Short-Wave_Radiation_Flux_surface_Mixed_intervals_Average  \
2017-06-01 09:00:00-07:00                                                0.0                    
2017-06-01 12:00:00-07:00                                                0.0                    
2017-06-01 15:00:00-07:00                                              200.0                    
2017-06-01 18:00:00-07:00                                              493.0                    
2017-06-01 21:00:00-07:00                                             1040.0                    

                           Temperature_surface  \
2017-06-01 09:00:00-07:00                293.5   
2017-06-01 12:00:00-07:00                292.0   
2017-06-01 15:00:00-07:00                304.5   
2017-06-01 18:00:00-07:00                316.0   
2017-06-01 21:00:00-07:00                317.5   

                           Total_cloud_cover_boundary_layer_cloud_Mixed_intervals_Average  \
2017-06-01 09:00:00-07:

In [9]:
data = model.get_processed_data(latitude, longitude, start, end)
print(data.head())

                            temp_air  wind_speed         ghi         dni  \
2017-06-01 09:00:00-07:00  20.350006    1.971243  258.775655   18.271178   
2017-06-01 12:00:00-07:00  18.850006    3.098709  382.713505   22.869109   
2017-06-01 15:00:00-07:00  31.350006    3.176948  541.312880  192.059614   
2017-06-01 18:00:00-07:00  42.850006    4.970010  148.700513  170.154591   
2017-06-01 21:00:00-07:00  44.350006    6.110720    0.000000    0.000000   

                                  dhi  total_clouds  low_clouds  mid_clouds  \
2017-06-01 09:00:00-07:00  245.973363          97.0         0.0         0.0   
2017-06-01 12:00:00-07:00  360.274059          96.0         0.0         0.0   
2017-06-01 15:00:00-07:00  386.760039          52.0         0.0         1.0   
2017-06-01 18:00:00-07:00  101.951364          56.0         0.0         1.0   
2017-06-01 21:00:00-07:00    0.000000           6.0         0.0         1.0   

                           high_clouds  
2017-06-01 09:00:00-07:00  

In [10]:
cloud_vars = ['total_clouds', 'low_clouds','mid_clouds', 'high_clouds']

In [14]:
data[cloud_vars].plot();
plt.ylabel('Cloud cover %');
plt.xlabel('Forecast Time ({})'.format(tz));
plt.legend();
plt.show()

In [16]:
from pvlib.pvsystem import PVSystem, retrieve_sam
from pvlib.tracking import SingleAxisTracker
from pvlib.modelchain import ModelChain

sandia_modules = retrieve_sam('sandiamod')
cec_inverters = retrieve_sam('cecinverter')
module = sandia_modules['Canadian_Solar_CS5P_220M___2009_']
inverter = cec_inverters['SMA_America__SC630CP_US_315V__CEC_2012_']

# model a big tracker for more fun
system = SingleAxisTracker(module_parameters=module,inverter_parameters=inverter,modules_per_string=15,strings_per_inverter=300)

# fx is a common abbreviation for forecast
fx_model = GFS()
fx_data = fx_model.get_processed_data(latitude, longitude, start, end)

# use a ModelChain object to calculate modeling intermediates
mc = ModelChain(system, fx_model.location)

# extract relevant data for model chain
mc.run_model(fx_data.index, weather=fx_data);

/Users/basilebouquet/Documents/Python/PV_FX/lib/python2.7/site-packages/pvlib/pvsystem.py:1303: RuntimeWarning: divide by zero encountered in log
  module['Voco'] + module['Cells_in_Series']*delta*np.log(Ee) +
/Users/basilebouquet/Documents/Python/PV_FX/lib/python2.7/site-packages/pvlib/pvsystem.py:1309: RuntimeWarning: divide by zero encountered in log
  module['C3']*module['Cells_in_Series']*((delta*np.log(Ee)) ** 2) +


In [ ]:
mc.ac.plot();
plt.ylim(0, None);
plt.ylabel('AC Power (W)');
plt.show()

In [ ]:
1+1